In [3]:
!ls

CeLEry-data-agumentation.ipynb	Mouse_sc_analysis.py
CellLocation4.py		Mouse_sc_analysis-results.py
CellLocation4_result2.py	Mouse_sc_pre.py
CellLocation4_result.py		Mouse_spotOTsc.ipynb
Mouse_CeLEry.ipynb		Mouse_Tangram.ipynb
Mouse_CeLEry_oldcode.ipynb	novosparc.ipynb
Mouse_novosparc.ipynb		Tangram4.py
Mouse_sc_analysis.ipynb		Untitled-Copy2.ipynb


## 1. Data Preperation

### 1.1 Preperation
Load packages:

In [4]:
import os,csv,re
import pandas as pd
import numpy as np
import scanpy as sc
import math
from skimage import io, color

from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import pickle

#Read original data and save it to h5ad
from scanpy import read_10x_h5
# import SpaGCN as spg

import json
import cv2 as cv

import anndata


In [5]:
%cd /lustre03/project/6075067/calcium/2021/CeLEry

/lustre03/project/6075067/calcium/2021/CeLEry


In [6]:
import sys
sys.path.append('/lustre03/project/6075067/calcium/2021/CeLEry/CeLEry_package')

import CeLEry as cel
cel.__version__

'1.1.2'

### 1.2 Load Data

In [7]:
MouseSC = sc.read("data/Seurat/MouseSC_scRNA_SeuratMouseSC.h5ad")

dataSection1full = sc.read("data/MouseBrain/MP1_sudo.h5ad")
genename = dataSection1full.var['genename']

In [8]:
ImageSec1=io.imread("data/MouseBrain/V1_Mouse_Brain_Sagittal_Posterior_image.tif")
ImageSec1sub = ImageSec1[3000:7000,6200:10500,:]
# spc.printimage (ImageSec1sub, "../output/CeLEry/imageselect")

imgray = cv.cvtColor(ImageSec1sub, cv.COLOR_BGR2GRAY)
imgray2 = imgray.copy()
imgray2[imgray2<160] = 0
imgray2[imgray2>160] = 255

## Take the subset of dataSection1
xcords = dataSection1full.obs["x"].to_numpy()
ycords = dataSection1full.obs["y"].to_numpy()

Section1Sub = dataSection1full[(xcords>=3000) & (xcords<7000) & (ycords>=6200) & (ycords<10500), MouseSC.var_names]
Section1Sub.obs = Section1Sub.obs/50
Section1Sub.obs = Section1Sub.obs.astype(int)
Section1Sub.obs["inner"] = 0

## Quality Control

for i in range(Section1Sub.obs.shape[0]):
    xi = Section1Sub.obs["x"][i]
    yi = Section1Sub.obs["y"][i]
    subarea = np.mean(imgray2[(xi*50-3000):(xi*50+50-3000), (yi*50-6200):(yi*50+50-6200)])
    if subarea<140 or xi*50>6000:
         Section1Sub.obs["inner"].iloc[i] = 1
    if yi*50>10200 or xi*50<1000:
         Section1Sub.obs["inner"].iloc[i] = 0

Section1Sub = Section1Sub[Section1Sub.obs["inner"] == 1, ]

In [9]:
## Calculating z-score
cel.get_zscore(Section1Sub)
cel.get_zscore(MouseSC)

In [10]:
Section1Sub

View of AnnData object with n_obs × n_vars = 5824 × 350
    obs: 'x', 'y', 'inner'
    var: 'genename'

In [11]:
MouseSC

AnnData object with n_obs × n_vars = 327723 × 350
    obs: 'exp_component_name', 'platform_label', 'cluster_color', 'cluster_order', 'cluster_label', 'class_color', 'class_order', 'class_label', 'subclass_color', 'subclass_order', 'subclass_label', 'full_genotype_color', 'full_genotype_id', 'full_genotype_label', 'sex_color', 'sex_id', 'donor_sex_label', 'region_color', 'region_id', 'region_label', 'cell_type_accession_color', 'cell_type_accession_id', 'cell_type_accession_label', 'cell_type_alias_color', 'cell_type_alias_id', 'cell_type_alias_label', 'cell_type_alt_alias_color', 'cell_type_alt_alias_id', 'cell_type_alt_alias_label', 'cell_type_designation_color', 'cell_type_designation_id', 'cell_type_designation_label', 'neighborhood_label', 'neighborhood_id', 'neighborhood_color', 'external_donor_name_color', 'external_donor_name_id', 'external_donor_name_label', 'facs_population_plan_color', 'facs_population_plan_id', 'facs_population_plan_label', 'injection_materials_color', 'inje

In [12]:
# selectsubcell = MouseSC.obs["subclass_label"].isin(["L2 IT ENTl", "L2 IT RHP", "L2/3 IT CTX-1", "L2/3 IT CTX-2", "L2/3 IT ENTl","L3 IT ENT", "L3 RSP-ACA"])
selectsubcell = anndata.concat([(MouseSC[MouseSC.obs["subclass_label"] == inds, :])[0:100,:] for inds in ["L2 IT ENTl", "L2 IT RHP", "L2/3 IT CTX-1", "L2/3 IT CTX-2", "L2/3 IT ENTl", "L3 IT ENT", "L3 RSP-ACA"]])

In [13]:
selectsubcell.obs

,exp_component_name,platform_label,cluster_color,cluster_order,cluster_label,class_color,class_order,class_label,subclass_color,subclass_order,...,injection_roi_color,injection_roi_id,injection_roi_label,injection_type_color,injection_type_id,injection_type_label,cortical_layer_label,outlier_call,outlier_type,n_counts
sample_name,,,,,,,,,,,,,,,,,,,,,
AAACCTGAGTCAATAG-L8TX_180406_01_C07,AAACCTGAGTCAATAG-8L8TX_180406_01_C07,10x,#0E8C30,139.0,139_L2 IT ENTl,#00ADEE,2.0,Glutamatergic,#0E8C30,11.0,...,#FF7373,1,NaN,#FF7373,1,NaN,0,False,nan,1178.0
AAACCTGCAACACCCG-L8TX_180406_01_H02,AAACCTGCAACACCCG-4L8TX_180406_01_H02,10x,#0E8C30,139.0,139_L2 IT ENTl,#00ADEE,2.0,Glutamatergic,#0E8C30,11.0,...,#FF7373,1,NaN,#FF7373,1,NaN,0,False,nan,3025.0
AAACCTGCACTAAGTC-L8TX_180406_01_E07,AAACCTGCACTAAGTC-9L8TX_180406_01_E07,10x,#0E8C30,139.0,139_L2 IT ENTl,#00ADEE,2.0,Glutamatergic,#0E8C30,11.0,...,#FF7373,1,NaN,#FF7373,1,NaN,0,False,nan,3101.0
AAACCTGCAGACAAGC-L8TX_180406_01_H02,AAACCTGCAGACAAGC-4L8TX_180406_01_H02,10x,#0E8C30,139.0,139_L2 IT ENTl,#00ADEE,2.0,Glutamatergic,#0E8C30,11.0,...,#FF7373,1,NaN,#FF7373,1,NaN,0,False,nan,3796.0
AAACCTGCATCGGGTC-L8TX_180406_01_E07,AAACCTGCATCGGGTC-9L8TX_180406_01_E07,10x,#0E8C30,139.0,139_L2 IT ENTl,#00ADEE,2.0,Glutamatergic,#0E8C30,11.0,...,#FF7373,1,NaN,#FF7373,1,NaN,0,False,nan,1529.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AACTCCCGTGTTGGGA-L8TX_180221_01_D11,AACTCCCGTGTTGGGA-3L8TX_180221_01_D11,10x,#5BC6A1,236.0,236_L3 RSP-ACA,#00ADEE,2.0,Glutamatergic,#5FCEA9,22.0,...,#FF7373,1,NaN,#FF7373,1,NaN,All,False,nan,2470.0
AACTCCCGTTACTGAC-L8TX_180221_01_B11,AACTCCCGTTACTGAC-1L8TX_180221_01_B11,10x,#5BC6A1,236.0,236_L3 RSP-ACA,#00ADEE,2.0,Glutamatergic,#5FCEA9,22.0,...,#FF7373,1,NaN,#FF7373,1,NaN,All,False,nan,4388.0
AACTCCCGTTGTCGCG-L8TX_180221_01_B11,AACTCCCGTTGTCGCG-1L8TX_180221_01_B11,10x,#4D9980,234.0,234_L3 RSP-ACA,#00ADEE,2.0,Glutamatergic,#5FCEA9,22.0,...,#FF7373,1,NaN,#FF7373,1,NaN,All,False,nan,1075.0


In [12]:
selectsubcell.obs.to_csv("output/CeLEry/Mousesc/Mouse_pheno_spaOTsc.csv")

### ------------------------------------------------------------------------------------------------------- ###
###        Perform spaOTsc analysis
### ------------------------------------------------------------------------------------------------------- ###


In [14]:
sys.path.append('/lustre03/project/6075067/calcium/2021/CeLEry/SpaOTsc_package')

from spaotsc import SpaOTsc

from scipy.spatial.distance import cdist

In [28]:
datatrain = Section1Sub
datatest = selectsubcell

In [29]:
S1_xmax = Section1Sub.obs['x'].max() + 1
S1_xmin = Section1Sub.obs['x'].min() - 1
S1_ymax = Section1Sub.obs['y'].max() + 1
S1_ymin = Section1Sub.obs['y'].min() - 1

random.seed(2021)
torch.manual_seed(2021)
np.random.seed(2021)
## Running spaOTsc
df_sc = pd.DataFrame(datatest.X)
is_dmat = cdist(datatrain.obs.to_numpy()[:,0:2], datatrain.obs.to_numpy()[:,0:2], 'minkowski', p=2.)
sc_dmat = cdist(datatest.X, datatest.X, 'minkowski', p=2.)
spsc = SpaOTsc.spatial_sc(sc_data=df_sc, is_dmat=is_dmat, sc_dmat=sc_dmat)
cost_matrix = cdist(datatest.X, datatrain.X, 'minkowski', p=2.)

map = spsc.transport_plan(cost_matrix)

In [30]:
map.shape

(700, 5824)

In [31]:
## Normalize the coordinates of both Sections
spx = (Section1Sub.obs.iloc[:,0] - S1_xmin) / (S1_xmax - S1_xmin)
spy = (Section1Sub.obs.iloc[:,1] - S1_ymin) / (S1_ymax - S1_ymin)

coords_predict_tangram = np.zeros((selectsubcell.obs.shape[0],2))
for i in range(map.shape[0]):
    bestindex = np.argmax(map[i,:])
    pred = torch.FloatTensor([spx[bestindex],spy[bestindex]])
    coords_predict_tangram[i,:] = pred


np.savetxt("{folder}/{name}_predmatrix.csv".format(folder = "output/CeLEry/Mousesc/", name = "spaOTsc_Mousesc"), coords_predict_tangram, delimiter=",")


### ------------------------------------------------------------------------------------------------------- ###
###        Perform novosparc analysis
### ------------------------------------------------------------------------------------------------------- ###


In [14]:
import novosparc as ns

datatrain = Section1Sub
datatest = selectsubcell

In [15]:
S1_xmax = Section1Sub.obs['x'].max() + 1
S1_xmin = Section1Sub.obs['x'].min() - 1
S1_ymax = Section1Sub.obs['y'].max() + 1
S1_ymin = Section1Sub.obs['y'].min() - 1

random.seed(2021)
torch.manual_seed(2021)
np.random.seed(2021)
### Running novosparc
locations_apriori = datatrain.obs[['x', 'y']].values
tissue = ns.cm.Tissue(dataset=datatest, locations=locations_apriori)
num_neighbors_s = num_neighbors_t = 5
    
# params for linear cost
atlas_genes = datatrain.var
markers = list(set(atlas_genes.values[:,0]))
num_genes = len(markers)
atlas_matrix = datatrain.to_df().values
markers_idx = pd.DataFrame({'markers_idx': np.arange(num_genes)}, index=markers)
markers_to_use = np.concatenate(markers_idx.loc[markers].values)

tissue.setup_reconstruction(atlas_matrix=atlas_matrix, 
                            markers_to_use=markers_to_use, 
                            num_neighbors_s=num_neighbors_s, 
                            num_neighbors_t=num_neighbors_t)
          
tissue.reconstruct(alpha_linear=0.8, epsilon=5e-3)

map = tissue.gw

Setting up for reconstruction ... done ( 33.2 seconds )
Reconstructing spatial information with 350 markers: 700 cells and 5824 locations ... 
Trying with epsilon: 5.00e-03


In [16]:
map.shape

(700, 5824)

In [17]:
## Normalize the coordinates of both Sections
spx = (Section1Sub.obs.iloc[:,0] - S1_xmin) / (S1_xmax - S1_xmin)
spy = (Section1Sub.obs.iloc[:,1] - S1_ymin) / (S1_ymax - S1_ymin)

coords_predict_tangram = np.zeros((selectsubcell.obs.shape[0],2))
for i in range(map.shape[0]):
    bestindex = np.argmax(map[i,:])
    pred = torch.FloatTensor([spx[bestindex],spy[bestindex]])
    coords_predict_tangram[i,:] = pred


np.savetxt("{folder}/{name}_predmatrix.csv".format(folder = "output/CeLEry/Mousesc/", name = "novosparc_Mousesc"), coords_predict_tangram, delimiter=",")
